# <div style="text-align: center; color: #1a5276;">Custom Layers</div>

## <font color='blue'>  Table of Contents </font>

1. [Introduction](#1)
2. [Setup](#2)
3. [Training](#3) <br>
    3.1. [Example 1](#3.1) <br>
    3.2. [Example 2](#3.2) <br>
4. [References](#references)

<a name="1"></a>
## <font color='blue'> 1. Introduction </font>

This notebook demonstrates how to build custom layers in PyTorch to extend model flexibility and improve performance. It covers:

- Implementing custom layers using nn.Module.

- Adding parameters and defining forward logic.

<a name="2"></a>
## <font color='blue'> 2. Setup </font>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.manual_seed(42)

<a name="3"></a>
## <font color='blue'> 3. Examples </font>

<a name="3.1"></a>
### <font color='#1f618d'> 3.1. Example 1 </font>

### Defining the custom layer

Here's a simple example of a custom PyTorch layer that performs a weighted sum of inputs followed by a non-linear activation:

<img src="images/CustomLinear.png"/>

In [5]:
class CustomLayer(nn.Module):
    """
    A custom PyTorch layer that applies a linear transformation 
    followed by a ReLU activation.

    Args:
        input_dim (int): Number of input features.
        output_dim (int): Number of output features.
    """
    def __init__(self, input_dim, output_dim):
        super(CustomLayer, self).__init__()
        
        # Learnable weight parameter for the linear transformation
        self.weight = nn.Parameter(torch.randn(input_dim, output_dim))
        
        # Learnable bias parameter initialized to zero
        self.bias = nn.Parameter(torch.zeros(output_dim))

    def forward(self, x):
        """
        Forward pass of the custom layer.
        
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim).
        
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, output_dim) 
                          after applying the linear transformation and ReLU.
        """
        x = torch.matmul(x, self.weight) + self.bias  # Linear transformation
        return F.relu(x)  # ReLU activation


This custom layer applies a linear transformation followed by a ReLU activation. Using nn.Parameter allows the layer's weights to be trainable. You can integrate this layer like any other PyTorch module.

### Using the custom layer

In [6]:
class SimpleModel(nn.Module):
    """
    A simple neural network that combines a custom layer with a standard linear layer.

    The first layer is a custom layer (`CustomLayer`) that applies a linear 
    transformation followed by ReLU. The second layer is a standard PyTorch 
    `nn.Linear` layer that outputs a single value, passed through a sigmoid 
    activation for binary classification.

    This demonstrates how to integrate custom layers into a PyTorch model.
    """
    def __init__(self):
        super(SimpleModel, self).__init__()
        
        # Custom layer: Transforms input features from 10 to 20 dimensions
        self.layer1 = CustomLayer(10, 20)
        
        # Standard linear layer: Transforms 20 features to 1 output
        self.layer2 = nn.Linear(20, 1)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, 10).
        
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, 1) with values 
                          in the range [0, 1] (after sigmoid activation).
        """
        x = self.layer1(x)  # Applying the custom layer
        return torch.sigmoid(self.layer2(x))  # Final output with sigmoid activation


In [7]:
# Testing the model
x = torch.randn(5, 10)  # Batch size 5, input size 10
model = SimpleModel()
output = model(x)
print(output)

tensor([[0.0847],
        [0.1793],
        [0.0875],
        [0.2268],
        [0.1330]], grad_fn=<SigmoidBackward0>)


<a name="3.2"></a>
### <font color='#1f618d'> 3.2. Example 2 </font>

This updated example includes batch normalization and dropout for improved training stability and regularization.

### Defining the custom layer

In [8]:
class CustomLayer(nn.Module):
    """
    A custom PyTorch layer that applies a linear transformation,
    batch normalization, ReLU activation, and dropout.

    Args:
        input_dim (int): Number of input features.
        output_dim (int): Number of output features.
        dropout_rate (float): Dropout rate for regularization (default is 0.3).
    """
    def __init__(self, input_dim, output_dim, dropout_rate=0.3):
        super(CustomLayer, self).__init__()
        
        # Learnable weight parameter for the linear transformation
        self.weight = nn.Parameter(torch.randn(input_dim, output_dim))
        
        # Learnable bias parameter initialized to zero
        self.bias = nn.Parameter(torch.zeros(output_dim))
        
        # Batch normalization to stabilize training
        self.bn = nn.BatchNorm1d(output_dim)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        """
        Forward pass of the custom layer.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, input_dim).
        
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, output_dim) 
                          after applying linear transformation, batch normalization, 
                          ReLU activation, and dropout.
        """
        x = torch.matmul(x, self.weight) + self.bias  # Linear transformation
        x = self.bn(x)  # Batch normalization
        x = F.relu(x)   # ReLU activation
        return self.dropout(x)  # Dropout for regularization


### Using the custom layer 

In [9]:
# Example usage
class SimpleModel(nn.Module):
    """
    A simple neural network that combines a custom layer with a standard linear layer.

    The first layer is a custom layer (`CustomLayer`) that applies a linear 
    transformation, batch normalization, ReLU activation, and dropout. 
    The second layer is a standard PyTorch `nn.Linear` layer that outputs a 
    single value, passed through a sigmoid activation for binary classification.

    This demonstrates how to integrate a custom layer into a PyTorch model.
    """
    def __init__(self):
        super(SimpleModel, self).__init__()
        
        # Custom layer: Transforms input features from 10 to 20 dimensions
        self.layer1 = CustomLayer(10, 20)
        
        # Standard linear layer: Transforms 20 features to 1 output
        self.layer2 = nn.Linear(20, 1)

    def forward(self, x):
        """
        Forward pass of the model.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, 10).
        
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, 1) with values 
                          in the range [0, 1] (after sigmoid activation).
        """
        x = self.layer1(x)  # Applying the custom layer
        return torch.sigmoid(self.layer2(x))  # Final output with sigmoid activation


In [10]:
# Testing the model
x = torch.randn(5, 10)  # Batch size 5, input size 10
model = SimpleModel()
output = model(x)
print(output)

tensor([[0.2799],
        [0.4136],
        [0.3648],
        [0.5205],
        [0.3727]], grad_fn=<SigmoidBackward0>)


<a name="references"></a>
## <font color='blue'> References </font>

[PyTorch Documentation](https://pytorch.org/docs/stable/index.html)